In [1]:
from pandas import read_csv
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape, 
                        activation=activation[0]))
    model.add(Dense(units=dense_units, activation=activation[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
 
demo_model = create_RNN(2, 1, (3,1), activation=['linear', 'linear'])

In [3]:
wx = demo_model.get_weights()[0] # Weights at previous
wh = demo_model.get_weights()[1] # Weights at current
bh = demo_model.get_weights()[2] # Bias at current
wy = demo_model.get_weights()[3] # Weights at next
by = demo_model.get_weights()[4] # Bias at next
 
print('wx = ', wx, ' wh = ', wh, ' bh = ', bh, ' wy =', wy, 'by = ', by)

wx =  [[-0.7582471   0.84121096]]  wh =  [[ 0.01967335 -0.9998064 ]
 [-0.9998064  -0.01967347]]  bh =  [0. 0.]  wy = [[ 0.19534087]
 [-0.66375417]] by =  [0.]


In [4]:
# Fit on Datapoints
x = np.array([1, 2, 3])

# Reshape the input to the required sample_size x time_steps x features 
x_input = np.reshape(x,(1, 3, 1))
y_pred_model = demo_model.predict(x_input) 

1/1 [==============================] - 0s 260ms/step


In [5]:
m = 2
h0 = np.zeros(m)
h1 = np.dot(x[0], wx) + h0 + bh
h2 = np.dot(x[1], wx) + np.dot(h1,wh) + bh
h3 = np.dot(x[2], wx) + np.dot(h2,wh) + bh
o3 = np.dot(h3, wy) + by 

In [6]:
print('h1 = ', h1,'h2 = ', h2,'h3 = ', h3)

h1 =  [[-0.75824708  0.84121096]] h2 =  [[-2.37245952  2.42397267]] h3 =  [[-4.74491885  4.84794516]]


In [7]:
print("Prediction from network ", y_pred_model)

Prediction from network  [[-4.1447206]]


In [8]:
print("Prediction from our computation ", o3)

Prediction from our computation  [[-4.14472038]]
